# Pre-processing Nebraska Allocation data for WaDEQA upload.
Date Updated: 08/26/2020
Purpose:  To pre-process the Nebraska data into one master file for simple DataFrame creation and extraction.
Joining API surface water data - to POD shapefile data via **RightID**.

In [1]:
#Needed Libararies
import os
import numpy as np
import pandas as pd
import requests
import json
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
#Working Directory and Input Files
workingDir = "C:/Users/rjame/Documents/WSWC Documents/MappingStatesDataToWaDE2.0/Nebraska/WaterAllocation/RawInputData"
os.chdir(workingDir)

In [3]:
#Retrieve POD shapefile data
sfInput = "ActiveSWPODs08242020_input.csv"
dfpodsftemp = pd.read_csv(sfInput)
dfpodsf = dfpodsftemp[['RightID', 'HUC_12', 'Longitude', 'Latitude']] #columns of interest
dfpodsf

,RightID,HUC_12,Longitude,Latitude
0,933,1.025000e+11,-101.964731,40.033267
1,945,1.025000e+11,-101.816246,40.035814
2,1501,1.025002e+11,-99.410350,40.005894
3,1504,1.025002e+11,-99.404383,40.009103
4,1505,1.025002e+11,-99.375045,40.013726
...,...,...,...,...
8786,3340,1.027020e+11,-97.146450,40.719856
8787,3453,1.027020e+11,-96.946811,40.474404
8788,13849,1.015001e+11,-98.484968,42.704781
8789,13850,1.024001e+11,-95.973326,40.086231


In [4]:
# Get all surface water points from NeDNR API.
# May get duplicatesd, should be roughly 8790 good rows to work with.
dfsw = pd.DataFrame()
checktimer = 1
page = 1

while (checktimer) < 11:
    url = "https://nednr.nebraska.gov/IwipApi/api/v1/WaterRights/AllSurfaceWaterPoints?page=" + str(page)
    page = 1 + page
    responseD = json.loads(requests.get(url).text)
    DLtL = responseD['Results']
    length = len(DLtL)
    for i in range(length):
        row = pd.DataFrame([DLtL[i]])
        dfsw = dfsw.append(row)
    checktimer = checktimer + 1

dfapisw = dfsw.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
dfapisw

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0,AF,0,None,130,Thayer,Little Blue,1.02702e+11,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None
0,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0,AF,0,None,20,Thayer,Little Blue,1.02702e+11,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None
0,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,1.02702e+11,1975-08-14T00:00:00,None,None,None,None,None,None,None
0,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,1.02702e+11,1975-08-14T00:00:00,None,None,None,None,None,None,None
0,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0,AF,0,None,314,Jefferson,Little Blue,1.02702e+11,1969-06-25T00:00:00,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,13307,Active,IR,A-11428C,NWNE S34-T30-R5W,16997,7,1,2F,None,Pump,None,Bazile Creek,0.07,CFS,31,70,5.1,Knox,Lewis & Clark,1.01701e+11,1968-05-22T00:00:00,None,1968-08-30T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder)
0,9398,Active,SI,A-15321,S26-T33-R1W,13225,1,1,2F,None,Pump,None,Hoesing Brothers Res.,5.7,AF,0,None,120,Cedar,Lewis & Clark,1.01701e+11,1978-08-15T00:00:00,None,1979-02-16T00:00:00,None,None,None,None,None
0,9427,Active,IR,A-2877,SWNW S20-T31-R1E,6947,1,1,2F,None,Pump,None,Norwegian Bow Creek,0.04,CFS,17,140,5,Cedar,Lewis & Clark,1.01701e+11,1938-06-17T00:00:00,None,1938-10-01T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder)
0,9455,Active,IR,A-8520,S5-T31-R1E,7001,1,1,2F,None,Pump,None,"Bow Creek, West",0.29,CFS,130,140,40,Cedar,Lewis & Clark,1.01701e+11,1956-04-26T00:00:00,None,1956-09-27T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder)


In [5]:
#Merging dataframes into one, using left-join.
df = pd.merge(dfapisw, dfpodsf, left_on='RightID', right_on='RightID', how='inner')
df

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0,AF,0,None,130,Thayer,Little Blue,1.02702e+11,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0,AF,0,None,20,Thayer,Little Blue,1.02702e+11,1985-06-04T00:00:00,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,1.02702e+11,1975-08-14T00:00:00,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979
3,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,1.02702e+11,1975-08-14T00:00:00,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979
4,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0,AF,0,None,314,Jefferson,Little Blue,1.02702e+11,1969-06-25T00:00:00,None,None,None,None,None,None,None,1.027021e+11,-97.358024,40.141120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9894,13307,Active,IR,A-11428C,NWNE S34-T30-R5W,16997,7,1,2F,None,Pump,None,Bazile Creek,0.07,CFS,31,70,5.1,Knox,Lewis & Clark,1.01701e+11,1968-05-22T00:00:00,None,1968-08-30T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.894716,42.528671
9895,9398,Active,SI,A-15321,S26-T33-R1W,13225,1,1,2F,None,Pump,None,Hoesing Brothers Res.,5.7,AF,0,None,120,Cedar,Lewis & Clark,1.01701e+11,1978-08-15T00:00:00,None,1979-02-16T00:00:00,None,None,None,None,None,1.017010e+11,-97.393170,42.812815
9896,9427,Active,IR,A-2877,SWNW S20-T31-R1E,6947,1,1,2F,None,Pump,None,Norwegian Bow Creek,0.04,CFS,17,140,5,Cedar,Lewis & Clark,1.01701e+11,1938-06-17T00:00:00,None,1938-10-01T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.345357,42.652380
9897,9455,Active,IR,A-8520,S5-T31-R1E,7001,1,1,2F,None,Pump,None,"Bow Creek, West",0.29,CFS,130,140,40,Cedar,Lewis & Clark,1.01701e+11,1956-04-26T00:00:00,None,1956-09-27T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.332456,42.696217


In [6]:
# Changing datatype of used date fields. 
df['PriorityDate'] = pd.to_datetime(df['PriorityDate'], errors = 'coerce')
df['PriorityDate'] = pd.to_datetime(df["PriorityDate"].dt.strftime('%m/%d/%Y'))
df.head(3)

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0,AF,0,None,130,Thayer,Little Blue,1.02702e+11,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0,AF,0,None,20,Thayer,Little Blue,1.02702e+11,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,1.02702e+11,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979


In [7]:
# Making Sure datatype of HUC12 is int.
def assignHUC12(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = ''
    else:
        outList = int(colrowValue)
    return outList

df['HUC12'] = df.apply(lambda row: assignHUC12(row['HUC12']), axis=1)
df.head(3)

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0,AF,0,None,130,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0,AF,0,None,20,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979


In [8]:
#Creating Beneficial Use from NeDWR Provdied code.  See metaData_SurfaceWaterWebSimpleSearch.pdf for details.

NebraskaBenUseCodeDict = {
"CO" : "Cooling",
"DG" : "Dredge",
"DI" : "Domestic, Irrigation and Manufacturing",
"DO" : "Domestic",
"DS" : "Domestic Storage",
"FC" : "Fish Culture",
"FL" : "Flood Control",
"FW" : "Fish and Wildlife",
"IF" : "Instream Flow",
"IG" : "Induced Ground Water Recharge",
"IN" : "Intentional Underground Storage",
"IR" : "Irrigation from Natural Stream",
"IS" : "Irrigation and Storage (an appropriation approved for both uses)",
"IU" : "Irrigation and Incidental Underground Storage",
"MF" : "Manufacturing",
"ML" : "Maintain Level of a Lake",
"MU" : "Municipal",
"OU" : "Storage Use-only for irrigation from reservoir on lands not covered by natural flow appropriation AND for Incidental Underground Storage",
"PI" : "Power and Incidental Underground Storage",
"PR" : "Power",
"PS" : "Supplemental Power and Incidental Underground Storage",
"PW" : "Public Water Supply",
"RC" : "Groundwater Recharge",
"RD" : "Raise Dam (for increase in head for power production)",
"SC" : "Supplemental Cooling",
"SD" : "Supplemental Domestic",
"SF" : "Supplemental Fish Culture",
"SI" : "Supplemental Irrigation",
"SO" : "Storage Use-only",
"SP" : "Supplemental Power",
"SS" : "Supplemental Storage",
"ST" : "Storage",
"SU" : "Storage and Incidental Underground Storage",
"TI" : "Temporary Transfer to In-Stream Use",
"UI" : "Supplemental Irrigation and Incidental Underground Storage",
"US" : "Incidental Underground Storage",
"WS" : "Waste Storage",
"WT" : "Wetlands"}

def assignRightUse(colrowValue):
    if colrowValue == '' or pd.isnull(colrowValue):
        outList = 'Unknown'
    else:
        String1 = colrowValue.strip()  # remove whitespace chars
        try:
            outList = NebraskaBenUseCodeDict[String1]
        except:
            outList = 'Unknown'
    return outList

df['BeneficialUseCategory'] = df.apply(lambda row: assignRightUse(row['RightUse']), axis=1)
df.head(3)

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude,BeneficialUseCategory
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0,AF,0,None,130,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041,Storage Use-only
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0,AF,0,None,20,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087,Storage Use-only
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only


In [9]:
# AllocationFlow_CFS - based on reporeted Unit
def assignAllocationFlow_CFS(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "CFS":
            outList = colvA
        else:
            outList = ''
    return outList

df['AllocationFlow_CFS'] = df.apply(lambda row: assignAllocationFlow_CFS(row['ProGrant'], row['Units']), axis=1)
df

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude,BeneficialUseCategory,AllocationFlow_CFS
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0,AF,0,None,130,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041,Storage Use-only,
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0,AF,0,None,20,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087,Storage Use-only,
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,
3,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,
4,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0,AF,0,None,314,Jefferson,Little Blue,102702070205,1969-06-25,None,None,None,None,None,None,None,1.027021e+11,-97.358024,40.141120,Storage Use-only,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9894,13307,Active,IR,A-11428C,NWNE S34-T30-R5W,16997,7,1,2F,None,Pump,None,Bazile Creek,0.07,CFS,31,70,5.1,Knox,Lewis & Clark,101701010704,1968-05-22,None,1968-08-30T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.894716,42.528671,Irrigation from Natural Stream,0.07
9895,9398,Active,SI,A-15321,S26-T33-R1W,13225,1,1,2F,None,Pump,None,Hoesing Brothers Res.,5.7,AF,0,None,120,Cedar,Lewis & Clark,101701011205,1978-08-15,None,1979-02-16T00:00:00,None,None,None,None,None,1.017010e+11,-97.393170,42.812815,Supplemental Irrigation,
9896,9427,Active,IR,A-2877,SWNW S20-T31-R1E,6947,1,1,2F,None,Pump,None,Norwegian Bow Creek,0.04,CFS,17,140,5,Cedar,Lewis & Clark,101701011102,1938-06-17,None,1938-10-01T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.345357,42.652380,Irrigation from Natural Stream,0.04
9897,9455,Active,IR,A-8520,S5-T31-R1E,7001,1,1,2F,None,Pump,None,"Bow Creek, West",0.29,CFS,130,140,40,Cedar,Lewis & Clark,101701011004,1956-04-26,None,1956-09-27T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.332456,42.696217,Irrigation from Natural Stream,0.29


In [10]:
# AllocationVolume_AF - based on reporeted Unit
def assignAllocationVolume_AF(colvA, colvB):
    if colvA == '' or pd.isnull(colvA):
        outList = ''
    else:
        colvB = colvB.strip()
        if colvB == "AF":
            outList = colvA
        else:
            outList = ''
    return outList

df['AllocationVolume_AF'] = df.apply(lambda row: assignAllocationVolume_AF(row['ProGrant'], row['Units']), axis=1)
df

,RightID,RightStatus,RightUse,ApplicationNumber,Location,PointOfDiversionID,PrimaryIndicator,PointofDiversionStatus,WaterDivision,FieldOffice,CarrierA,CarrierB,SourceName,ProGrant,Units,GPM,Rate,CurrentTotalAcres,CountyName,NRDName,HUC12,PriorityDate,BeneficialDate,ApprovalDate,IrrigationProject,Notice_Status,Notice_EffectiveDate,Notice_Type,Reason_AdminAction,HUC_12,Longitude,Latitude,BeneficialUseCategory,AllocationFlow_CFS,AllocationVolume_AF
0,2472,Active,SO,A-16395,S34-T1-R2W,10443,1,1,1C,None,Pump,,Mumm Reservoir,0,AF,0,None,130,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.522380,40.011041,Storage Use-only,,0
1,2476,Active,SO,A-16394,S19-T1-R1W,10442,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 2-A,0,AF,0,None,20,Thayer,Little Blue,102702070203,1985-06-04,1987-10-01T00:00:00,1985-07-15T00:00:00,None,None,None,None,None,1.027021e+11,-97.467746,40.034087,Storage Use-only,,0
2,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10386,7,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,,0
3,2478,Active,SO,A-13735,SWNE S2-T1-R2W,10387,1,1,1C,None,Pump,,Bowman Spring Branch Reservoir No. 3-E,0,AF,0,None,75,Thayer,Little Blue,102702070203,1975-08-14,None,None,None,None,None,None,None,1.027021e+11,-97.512848,40.076979,Storage Use-only,,0
4,2510,Active,SO,A-11738,S18-T2-R1E,10376,1,1,1C,None,Pump,,Buckley Creek Reservoir 3-C,0,AF,0,None,314,Jefferson,Little Blue,102702070205,1969-06-25,None,None,None,None,None,None,None,1.027021e+11,-97.358024,40.141120,Storage Use-only,,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9894,13307,Active,IR,A-11428C,NWNE S34-T30-R5W,16997,7,1,2F,None,Pump,None,Bazile Creek,0.07,CFS,31,70,5.1,Knox,Lewis & Clark,101701010704,1968-05-22,None,1968-08-30T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.894716,42.528671,Irrigation from Natural Stream,0.07,
9895,9398,Active,SI,A-15321,S26-T33-R1W,13225,1,1,2F,None,Pump,None,Hoesing Brothers Res.,5.7,AF,0,None,120,Cedar,Lewis & Clark,101701011205,1978-08-15,None,1979-02-16T00:00:00,None,None,None,None,None,1.017010e+11,-97.393170,42.812815,Supplemental Irrigation,,5.7
9896,9427,Active,IR,A-2877,SWNW S20-T31-R1E,6947,1,1,2F,None,Pump,None,Norwegian Bow Creek,0.04,CFS,17,140,5,Cedar,Lewis & Clark,101701011102,1938-06-17,None,1938-10-01T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.345357,42.652380,Irrigation from Natural Stream,0.04,
9897,9455,Active,IR,A-8520,S5-T31-R1E,7001,1,1,2F,None,Pump,None,"Bow Creek, West",0.29,CFS,130,140,40,Cedar,Lewis & Clark,101701011004,1956-04-26,None,1956-09-27T00:00:00,None,Open,2018-05-02T00:00:00,Open,Natural Flow (Reminder),1.017010e+11,-97.332456,42.696217,Irrigation from Natural Stream,0.29,


In [11]:
# Exporting output files.
df.to_csv('P_NebraskaMaster.csv', index=False)  # The output.